# 開始爬蟲

In [8]:
from time import sleep
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
# 搜尋目前上映那些電影，擷取出其ID資訊
name_list = []
author_list = []
intro_list = []
type_list = []
for i in range(200):
    s = str(i).zfill(4)
    url = "https://reading.udn.com/v2/bookDesc.do?id=18" + s
    r = requests.get(url)
    web_content = r.text
    soup = BeautifulSoup(web_content, 'lxml')
    book_name = soup.find('h2')
    print(i)
    if(book_name != None):
        sleep(5)
        #print("書籍名稱:"+book_name.text.strip())
        name_list.append(book_name.text.strip())
        book_author1 = soup.find('div', class_ ="info_centerbox_line")
        book_author = book_author1.find('h4')
        author_list.append(book_author.text.strip())
        #print("作者:"+book_author.text.strip())
        book_intro1 = soup.find('div', class_ ="book_intro")
        book_intro = book_intro1.find('p')
        intro_list.append(book_intro.text.strip())
        #print("劇情簡介:"+book_intro.text.strip())
        book_type1 = soup.find('div', class_= "path")
        book_type = book_type1.find_next('a').find_next('a').find_next('a')
        type_list.append(book_type.text.strip())
        #print("分類:"+book_type.text.strip())
df_book_id = pd.DataFrame()
df_book_id["書籍名稱"]=name_list
df_book_id["作者"]=author_list
df_book_id["劇情簡介"]=intro_list
df_book_id["分類"]=type_list
#存檔案
df_book_id.to_csv("./final.csv",encoding="utf-8-sig")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


# 訓練

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import jieba
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
import jieba.analyse

In [7]:
data = pd.read_csv("final.csv",encoding = 'utf_8_sig')
data.head()
row,col = data.shape
temp=[]
temp=data['劇情簡介']
for i in range(row):
    temp[i] = str(temp[i])
data['劇情簡介']=temp
data

<ipython-input-7-852ebea1497c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp[i] = str(temp[i])


,Unnamed: 0,書籍名稱,作者,劇情簡介,分類
0,0,到老師府辦桌：台北老家族的陳家菜,陳玠甫,台灣成長、美國留學、兩岸生活\r 擁有老靈魂的現代說書人\r 用一年的餐桌料理為您展示：...,生活休閒
1,1,從零開始：一段從失去中，重新找到家與愛的旅程,滕碧．洛克,沒有任何人事能治癒失去摯愛的傷，還有什麼能夠重新帶來生命的力量？一個觸動靈魂，論及悲傷探索、...,心靈勵志
2,2,太宰治請留步,黃文鉅,倘若你覺得自己三衰七敗，看不見明天的太陽，千萬、千萬不要急著去送死……痛必無獨，衰也有伴厭世...,文學
3,3,淡軼,文生,33個故事、33種人生，33也是耶穌死亡的年紀，但這數字同時也有種特別的味道。\r 整個世界...,文學
4,4,征服職場 Day by Day：小桌曆英文超給力系列,賴世雄教學團隊,nan,語言學習
...,...,...,...,...,...
162,162,那位神明是抖S,むつきらん,nan,圖文漫畫
163,163,隱密的性愛 inside sex,七ノ日,nan,圖文漫畫
164,164,Sugar ＆ Mustard－砂糖與黃芥末－,はなさわ浪雄,nan,圖文漫畫
165,165,寂寞難耐的抱枕,白松,nan,圖文漫畫


In [8]:
data.to_csv('Final_new.csv', encoding='utf_8_sig' , index = False)

In [11]:
def content_cut(text):
    #specialChars = "！#$%^&*（）《  》【]「"
    #text = text.replace("★" , "").replace("\r\n" , "").replace("！","").replace("《","").replace("》","").replace("，","").replace("/","").replace("（","").replace("）","")
    return " ".join(list(jieba.analyse.extract_tags(text,topK=2)))

data['分詞劇情'] = data['劇情簡介'].map(content_cut) 

In [12]:
data = { 'type':data["分類"] ,'name':data["書籍名稱"],'article':data['分詞劇情']}
df = pd.DataFrame(data)

df['key'] = df['name'].astype(str)+' '+df['type'].astype(str)+' '+df['article'].astype(str)

# 精準度

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
def count(df): 
    tfidf2 = TfidfVectorizer()
    re = tfidf2.fit_transform(df['key'])
    return re

def knn(X,Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y['type'].str[0:2], test_size=0.2)
    clf=KNeighborsClassifier(n_neighbors=51)
    clf.fit(X_train,y_train)

    y_pred = clf.predict(X_test)
    y_test=y_test.values

    print("Test set score:",metrics.accuracy_score(y_test, y_pred))
tf = count(df)
knn(tf,df)

Test set score: 0.4411764705882353
